In [1]:
import pandas as pd

In [2]:
df_NWHL = pd.read_csv(f"Rotman MMA Summer Datathon NWHL.csv")
df_NWHL.head()

,game_date,Home Team,Away Team,Period,Clock,Home Team Skaters,Away Team Skaters,Home Team Goals,Away Team Goals,Team,...,Event,X Coordinate,Y Coordinate,Detail 1,Detail 2,Detail 3,Detail 4,Player 2,X Coordinate 2,Y Coordinate 2
0,2021-01-23,Minnesota Whitecaps,Boston Pride,1,20:00,5,5,0,0,Boston Pride,...,Faceoff Win,100,43,Backhand,NaN,NaN,NaN,Stephanie Anderson,NaN,NaN
1,2021-01-23,Minnesota Whitecaps,Boston Pride,1,19:58,5,5,0,0,Boston Pride,...,Puck Recovery,107,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-01-23,Minnesota Whitecaps,Boston Pride,1,19:57,5,5,0,0,Boston Pride,...,Zone Entry,125,28,Carried,NaN,NaN,NaN,Maddie Rowe,NaN,NaN
3,2021-01-23,Minnesota Whitecaps,Boston Pride,1,19:55,5,5,0,0,Boston Pride,...,Shot,131,28,Snapshot,On Net,t,f,NaN,NaN,NaN
4,2021-01-23,Minnesota Whitecaps,Boston Pride,1,19:53,5,5,0,0,Boston Pride,...,Faceoff Win,169,21,Backhand,NaN,NaN,NaN,Stephanie Anderson,NaN,NaN


In [3]:
df_NWHL.columns

Index(['game_date', 'Home Team', 'Away Team', 'Period', 'Clock',
       'Home Team Skaters', 'Away Team Skaters', 'Home Team Goals',
       'Away Team Goals', 'Team', 'Player', 'Event', 'X Coordinate',
       'Y Coordinate', 'Detail 1', 'Detail 2', 'Detail 3', 'Detail 4',
       'Player 2', 'X Coordinate 2', 'Y Coordinate 2'],
      dtype='object')

In [4]:
df_player_list = df_NWHL.groupby(["Team", "Player"]).size().reset_index().rename(columns={0:"count"})
player_list = df_player_list.Player.tolist()

In [5]:
df_player_list.head()

,Team,Player,count
0,Boston Pride,Briana Mastel,244
1,Boston Pride,Carlee Turner,251
2,Boston Pride,Christina Putigna,365
3,Boston Pride,Jenna Rheault,73
4,Boston Pride,Jillian Dempsey,336


In [6]:
df_player_list["Player"].value_counts().unique()

array([1], dtype=int64)

In [7]:
#df.shape[0]

In [8]:
def shooting_stats(player_name, df):
    df_attendence = df.groupby(["game_date"]).apply(lambda x: x[(x['Player']==player_name)]['Event'].count()).to_frame(name='attendence')
    not_played_dates = df_attendence[df_attendence['attendence']==0].index.to_list()
    df_shot = df.groupby(["game_date"]).apply(lambda x: x[(x['Event']=='Shot') & (x['Player']==player_name)]['Event'].count()).to_frame(name='shot_number')
    df_shot = df_shot.drop(not_played_dates)
    df_goal = df.groupby(["game_date"]).apply(lambda x: x[(x['Event']=='Goal') & (x['Player']==player_name)]['Event'].count()).to_frame(name='goal_number')
    df_goal = df_goal.drop(not_played_dates)
    df_scoring_merged = df_shot.join(df_goal, how="left")
    num_games_played = len(df_scoring_merged)
    df_scoring_merged["total_shot_number"] = df_scoring_merged["shot_number"] + df_scoring_merged["goal_number"]
    df_scoring_merged["shot_percentage"] = df_scoring_merged["goal_number"]/df_scoring_merged["total_shot_number"]
    
    total_shots_made = df_scoring_merged["total_shot_number"].sum()
    total_goals_made = df_scoring_merged["goal_number"].sum()
    average_shots_made = df_scoring_merged["total_shot_number"].mean()
    average_shots_percentage = df_scoring_merged["shot_percentage"].mean()
    overall_shots_percentage = (df_scoring_merged["goal_number"].sum())/(df_scoring_merged["total_shot_number"].sum())
    
    player_shooting_stats_dict = {
        "name": player_name,
        "num_games_played": num_games_played,
        "total_shots_made": total_shots_made,
        "total_goals_made": total_goals_made,
        "average_shots_made": average_shots_made,
        "average_shots_percentage": average_shots_percentage,
        "overall_shots_percentage": overall_shots_percentage
    }
    
    return player_shooting_stats_dict

In [9]:
# shooting_stats_list

In [10]:
len(player_list)

126

In [11]:
def passing_stats(player_name, df):
    df_attendence = df.groupby(["game_date"]).apply(lambda x: x[(x['Player']==player_name)]['Event'].count()).to_frame(name='attendence')
    not_played_dates = df_attendence[df_attendence['attendence']==0].index.to_list()
    df_play = df.groupby(["game_date"]).apply(lambda x: x[(x['Event']=='Play') & (x['Player']==player_name)]['Event'].count()).to_frame(name='play_number')
    df_play = df_play.drop(not_played_dates)
    df_incom_play = df.groupby(["game_date"]).apply(lambda x: x[(x['Event']=='Incomplete Play') & (x['Player']==player_name)]['Event'].count()).to_frame(name='incom_play_number')
    df_incom_play = df_incom_play.drop(not_played_dates)
    df_passing_merged = df_incom_play.join(df_play, how="left")
    
    num_games_played = len(df_passing_merged)
    
    df_passing_merged["total_pass_number"] = df_passing_merged["play_number"] + df_passing_merged["incom_play_number"]
    df_passing_merged["pass_percentage"] = df_passing_merged["play_number"]/df_passing_merged["total_pass_number"]
    
    total_passes_made = df_passing_merged["total_pass_number"].sum()
    total_plays_made = df_passing_merged["play_number"].sum()
    average_passes_made = df_passing_merged["total_pass_number"].mean()
    average_passes_percentage = df_passing_merged["pass_percentage"].mean()
    overall_passes_percentage = (df_passing_merged["play_number"].sum())/(df_passing_merged["total_pass_number"].sum())
    

    
    assist_dict = dict()
    df_len = len(df)

    for i in range(len(df)):
        if i == df_len-1:
            pass
        elif (df.loc[i, "Player"] == player_name) & (df.loc[i, "Event"] == "Play"):
            if (df.loc[i+1, "Event"] == "Goal") & (df.loc[i, "Team"] == df.loc[i+1, "Team"]):
                assist_dict[str(df.loc[i, "game_date"]) + "_" + str(df.loc[i, "Clock"]) + "_" + str(df.loc[i+1, "Player"])] = "passer " + df.loc[i, "Player"]
    
    total_assists = len(assist_dict)
    
    player_passing_stats_dict = {
        "name": player_name,
        "num_games_played": num_games_played,
        "total_passes_made": total_passes_made,
        "total_plays_made": total_plays_made,
        "average_passes_made": average_passes_made,
        "average_passes_percentage": average_passes_percentage,
        "overall_passes_percentage": overall_passes_percentage,
        "total_assists": total_assists
    }
    return player_passing_stats_dict

C:\Users\ddjgy\AppData\Local\Temp\ipykernel_4448\1004927161.py:17: RuntimeWarning: invalid value encountered in longlong_scalars
  overall_shots_percentage = (df_scoring_merged["goal_numebr"].sum())/(df_scoring_merged["total_shot_number"].sum())
C:\Users\ddjgy\AppData\Local\Temp\ipykernel_4448\1004927161.py:17: RuntimeWarning: invalid value encountered in longlong_scalars
  overall_shots_percentage = (df_scoring_merged["goal_numebr"].sum())/(df_scoring_merged["total_shot_number"].sum())
C:\Users\ddjgy\AppData\Local\Temp\ipykernel_4448\1004927161.py:17: RuntimeWarning: invalid value encountered in longlong_scalars
  overall_shots_percentage = (df_scoring_merged["goal_numebr"].sum())/(df_scoring_merged["total_shot_number"].sum())
C:\Users\ddjgy\AppData\Local\Temp\ipykernel_4448\1004927161.py:17: RuntimeWarning: invalid value encountered in longlong_scalars
  overall_shots_percentage = (df_scoring_merged["goal_numebr"].sum())/(df_scoring_merged["total_shot_number"].sum())
C:\Users\ddjgy\A

In [27]:
df = df_NWHL.copy()
player_name = "Taylor Wenczkowski"

In [37]:
df_attendence = df.groupby(["game_date"]).apply(lambda x: x[(x['Player']==player_name)]['Event'].count()).to_frame(name='attendence')
not_played_dates = df_attendence[df_attendence['attendence']==0].index.to_list()
df_dump_in = df.groupby(["game_date"]).apply(lambda x: x[((x['Event']=='Dump In/Out') & (x['Player']==player_name)) & (( x['X Coordinate'] > 75) & ( x['X Coordinate'] < 125))]["Detail 1"].count()).to_frame(name='dump_in_number')
df_dump_in = df_dump_in.drop(not_played_dates)

In [38]:
df_dump_in

,dump_in_numebr
game_date,
2021-01-23,1
2021-01-24,1
2021-01-26,0
2021-01-27,1
2021-01-30,4
2021-01-31,0
2021-02-01,2


In [53]:
df_len = len(df)

df_attendence = df.groupby(["game_date"]).apply(lambda x: x[(x['Player']==player_name)]['Event'].count()).to_frame(name='attendence')
not_played_dates = df_attendence[df_attendence['attendence']==0].index.to_list()


dump_dict = dict()

for i in range(len(df)):
    game_date = df.loc[i, "game_date"]
    if game_date in not_played_dates:
        pass
    elif game_date in dump_dict:
        if (df.loc[i, 'Event']=='Dump In/Out') & (df.loc[i, 'Player']==player_name):
            if 75 < df.loc[i, 'X Coordinate'] < 125:
                if df.loc[i, "Detail 1"] == 'Retained':
                    dump_dict[game_date]["number_of_retained_dump_in"] += 1
                elif df.loc[i, "Detail 1"] == 'Lost':
                    dump_dict[game_date]["number_of_lost_dump_in"] += 1
            elif df.loc[i, 'X Coordinate'] < 75:
                if df.loc[i, "Detail 1"] == 'Retained':
                    dump_dict[game_date]["number_of_retained_dump_out"] += 1
                elif df.loc[i, "Detail 1"] == 'Lost':
                    dump_dict[game_date]["number_of_lost_dump_out"] += 1
                    
    elif game_date not in dump_dict:
        dump_dict[game_date] = dict()
        dump_dict[game_date]["number_of_retained_dump_in"] = 0
        dump_dict[game_date]["number_of_lost_dump_in"] = 0
        dump_dict[game_date]["number_of_retained_dump_out"] = 0
        dump_dict[game_date]["number_of_lost_dump_out"] = 0
        
df_dumping = pd.DataFrame.from_dict(dump_dict).T
num_games_played = len(df_dumping)

total_dump_ins_made = df_dumping["number_of_retained_dump_in"].sum() + df_dumping["number_of_lost_dump_in"].sum()
total_dump_outs_made = df_dumping["number_of_retained_dump_out"].sum() + df_dumping["number_of_lost_dump_out"].sum()

average_dump_ins_made = (df_dumping["number_of_retained_dump_in"] + df_dumping["number_of_lost_dump_in"]).mean()
average_dump_outs_made = (df_dumping["number_of_retained_dump_out"] + df_dumping["number_of_lost_dump_out"]).mean()

average_dump_ins_percentage = (df_dumping["number_of_retained_dump_in"]/(df_dumping["number_of_retained_dump_in"] + df_dumping["number_of_lost_dump_in"])).mean()
average_dump_outs_percentage = (df_dumping["number_of_retained_dump_out"]/(df_dumping["number_of_retained_dump_out"] + df_dumping["number_of_lost_dump_out"])).mean()

overall_dump_ins_percentage = (df_dumping["number_of_retained_dump_in"].sum())/(df_dumping["number_of_lost_dump_in"].sum())
overall_dump_outs_percentage = (df_dumping["number_of_retained_dump_out"].sum())/(df_dumping["number_of_lost_dump_out"].sum())

player_passing_stats_dict = {
    "name": player_name,
    "num_games_played": num_games_played,
    "total_dump_ins_made": total_dump_ins_made,
    "average_dump_ins_made": average_dump_ins_made,
    "average_dump_ins_percentage": average_dump_ins_percentage,
    "overall_dump_ins_percentage": overall_dump_ins_percentage,
    
    "total_dump_outs_made": total_dump_outs_made,
    "average_dump_outs_made": average_dump_outs_made,
    "average_dump_outs_percentage": average_dump_outs_percentage,
    "overall_dump_outs_percentage": overall_dump_outs_percentage,
    
}

In [50]:
dump_dict

{'2021-01-23': {'number_of_retained_dump_in': 1,
  'number_of_lost_dump_in': 0,
  'number_of_retained_dump_out': 1,
  'number_of_lost_dump_out': 3},
 '2021-01-24': {'number_of_retained_dump_in': 0,
  'number_of_lost_dump_in': 1,
  'number_of_retained_dump_out': 0,
  'number_of_lost_dump_out': 1},
 '2021-01-26': {'number_of_retained_dump_in': 0,
  'number_of_lost_dump_in': 0,
  'number_of_retained_dump_out': 0,
  'number_of_lost_dump_out': 0},
 '2021-01-27': {'number_of_retained_dump_in': 0,
  'number_of_lost_dump_in': 1,
  'number_of_retained_dump_out': 1,
  'number_of_lost_dump_out': 0},
 '2021-01-30': {'number_of_retained_dump_in': 0,
  'number_of_lost_dump_in': 4,
  'number_of_retained_dump_out': 0,
  'number_of_lost_dump_out': 1},
 '2021-01-31': {'number_of_retained_dump_in': 0,
  'number_of_lost_dump_in': 0,
  'number_of_retained_dump_out': 0,
  'number_of_lost_dump_out': 0},
 '2021-02-01': {'number_of_retained_dump_in': 0,
  'number_of_lost_dump_in': 2,
  'number_of_retained_dump

In [52]:
pd.DataFrame.from_dict(dump_dict).T

,number_of_retained_dump_in,number_of_lost_dump_in,number_of_retained_dump_out,number_of_lost_dump_out
2021-01-23,1,0,1,3
2021-01-24,0,1,0,1
2021-01-26,0,0,0,0
2021-01-27,0,1,1,0
2021-01-30,0,4,0,1
2021-01-31,0,0,0,0
2021-02-01,0,2,2,1


In [54]:
player_passing_stats_dict

{'name': 'Taylor Wenczkowski',
 'num_games_played': 7,
 'total_dump_ins_made': 9,
 'average_dump_ins_made': 1.2857142857142858,
 'average_dump_ins_percentage': 0.2,
 'overall_dump_ins_percentage': 0.125,
 'total_dump_outs_made': 10,
 'average_dump_outs_made': 1.4285714285714286,
 'average_dump_outs_percentage': 0.3833333333333333,
 'overall_dump_outs_percentage': 0.6666666666666666}

In [57]:
def dumping_stats(player_name, df):
    df_len = len(df)

    df_attendence = df.groupby(["game_date"]).apply(lambda x: x[(x['Player']==player_name)]['Event'].count()).to_frame(name='attendence')
    not_played_dates = df_attendence[df_attendence['attendence']==0].index.to_list()


    dump_dict = dict()

    for i in range(len(df)):
        game_date = df.loc[i, "game_date"]
        if game_date in not_played_dates:
            pass
        elif game_date in dump_dict:
            if (df.loc[i, 'Event']=='Dump In/Out') & (df.loc[i, 'Player']==player_name):
                if 75 < df.loc[i, 'X Coordinate'] < 125:
                    if df.loc[i, "Detail 1"] == 'Retained':
                        dump_dict[game_date]["number_of_retained_dump_in"] += 1
                    elif df.loc[i, "Detail 1"] == 'Lost':
                        dump_dict[game_date]["number_of_lost_dump_in"] += 1
                elif df.loc[i, 'X Coordinate'] < 75:
                    if df.loc[i, "Detail 1"] == 'Retained':
                        dump_dict[game_date]["number_of_retained_dump_out"] += 1
                    elif df.loc[i, "Detail 1"] == 'Lost':
                        dump_dict[game_date]["number_of_lost_dump_out"] += 1

        elif game_date not in dump_dict:
            dump_dict[game_date] = dict()
            dump_dict[game_date]["number_of_retained_dump_in"] = 0
            dump_dict[game_date]["number_of_lost_dump_in"] = 0
            dump_dict[game_date]["number_of_retained_dump_out"] = 0
            dump_dict[game_date]["number_of_lost_dump_out"] = 0

    df_dumping = pd.DataFrame.from_dict(dump_dict).T
    num_games_played = len(df_dumping)

    total_dump_ins_made = df_dumping["number_of_retained_dump_in"].sum() + df_dumping["number_of_lost_dump_in"].sum()
    total_dump_outs_made = df_dumping["number_of_retained_dump_out"].sum() + df_dumping["number_of_lost_dump_out"].sum()

    average_dump_ins_made = (df_dumping["number_of_retained_dump_in"] + df_dumping["number_of_lost_dump_in"]).mean()
    average_dump_outs_made = (df_dumping["number_of_retained_dump_out"] + df_dumping["number_of_lost_dump_out"]).mean()

    average_dump_ins_percentage = (df_dumping["number_of_retained_dump_in"]/(df_dumping["number_of_retained_dump_in"] + df_dumping["number_of_lost_dump_in"])).mean()
    average_dump_outs_percentage = (df_dumping["number_of_retained_dump_out"]/(df_dumping["number_of_retained_dump_out"] + df_dumping["number_of_lost_dump_out"])).mean()

    overall_dump_ins_percentage = (df_dumping["number_of_retained_dump_in"].sum())/(df_dumping["number_of_lost_dump_in"].sum())
    overall_dump_outs_percentage = (df_dumping["number_of_retained_dump_out"].sum())/(df_dumping["number_of_lost_dump_out"].sum())

    player_passing_stats_dict = {
        "name": player_name,
        "num_games_played": num_games_played,
        "total_dump_ins_made": total_dump_ins_made,
        "average_dump_ins_made": average_dump_ins_made,
        "average_dump_ins_percentage": average_dump_ins_percentage,
        "overall_dump_ins_percentage": overall_dump_ins_percentage,

        "total_dump_outs_made": total_dump_outs_made,
        "average_dump_outs_made": average_dump_outs_made,
        "average_dump_outs_percentage": average_dump_outs_percentage,
        "overall_dump_outs_percentage": overall_dump_outs_percentage,

    }
    
    return player_passing_stats_dict

In [58]:
passing_stats_list = list()
shooting_stats_list = list()
dumping_stats_list = list()
for player in player_list:
    player_shooting_stats = shooting_stats(player, df_NWHL.copy())
    shooting_stats_list.append(player_shooting_stats)
    player_passing_stats = passing_stats(player, df_NWHL.copy())
    passing_stats_list.append(player_passing_stats)
    player_dumping_stats = dumping_stats(player, df_NWHL.copy())
    dumping_stats_list.append(player_dumping_stats)


C:\Users\ddjgy\AppData\Local\Temp\ipykernel_4448\1004927161.py:17: RuntimeWarning: invalid value encountered in longlong_scalars
  overall_shots_percentage = (df_scoring_merged["goal_numebr"].sum())/(df_scoring_merged["total_shot_number"].sum())
C:\Users\ddjgy\AppData\Local\Temp\ipykernel_4448\3461801813.py:46: RuntimeWarning: invalid value encountered in longlong_scalars
  overall_dump_ins_percentage = (df_dumping["number_of_retained_dump_in"].sum())/(df_dumping["number_of_lost_dump_in"].sum())
C:\Users\ddjgy\AppData\Local\Temp\ipykernel_4448\3461801813.py:47: RuntimeWarning: invalid value encountered in longlong_scalars
  overall_dump_outs_percentage = (df_dumping["number_of_retained_dump_out"].sum())/(df_dumping["number_of_lost_dump_out"].sum())
C:\Users\ddjgy\AppData\Local\Temp\ipykernel_4448\1004927161.py:17: RuntimeWarning: invalid value encountered in longlong_scalars
  overall_shots_percentage = (df_scoring_merged["goal_numebr"].sum())/(df_scoring_merged["total_shot_number"].su

C:\Users\ddjgy\AppData\Local\Temp\ipykernel_4448\3461801813.py:46: RuntimeWarning: invalid value encountered in longlong_scalars
  overall_dump_ins_percentage = (df_dumping["number_of_retained_dump_in"].sum())/(df_dumping["number_of_lost_dump_in"].sum())
C:\Users\ddjgy\AppData\Local\Temp\ipykernel_4448\1004927161.py:17: RuntimeWarning: invalid value encountered in longlong_scalars
  overall_shots_percentage = (df_scoring_merged["goal_numebr"].sum())/(df_scoring_merged["total_shot_number"].sum())
C:\Users\ddjgy\AppData\Local\Temp\ipykernel_4448\3461801813.py:46: RuntimeWarning: invalid value encountered in longlong_scalars
  overall_dump_ins_percentage = (df_dumping["number_of_retained_dump_in"].sum())/(df_dumping["number_of_lost_dump_in"].sum())
C:\Users\ddjgy\AppData\Local\Temp\ipykernel_4448\3461801813.py:46: RuntimeWarning: invalid value encountered in longlong_scalars
  overall_dump_ins_percentage = (df_dumping["number_of_retained_dump_in"].sum())/(df_dumping["number_of_lost_dump_i

In [59]:
dumping_stats_list

[{'name': 'Briana Mastel',
  'num_games_played': 5,
  'total_dump_ins_made': 5,
  'average_dump_ins_made': 1.0,
  'average_dump_ins_percentage': 0.1111111111111111,
  'overall_dump_ins_percentage': 0.25,
  'total_dump_outs_made': 13,
  'average_dump_outs_made': 2.6,
  'average_dump_outs_percentage': 0.15,
  'overall_dump_outs_percentage': 0.18181818181818182},
 {'name': 'Carlee Turner',
  'num_games_played': 7,
  'total_dump_ins_made': 3,
  'average_dump_ins_made': 0.42857142857142855,
  'average_dump_ins_percentage': 0.0,
  'overall_dump_ins_percentage': 0.0,
  'total_dump_outs_made': 16,
  'average_dump_outs_made': 2.2857142857142856,
  'average_dump_outs_percentage': 0.11904761904761904,
  'overall_dump_outs_percentage': 0.14285714285714285},
 {'name': 'Christina Putigna',
  'num_games_played': 7,
  'total_dump_ins_made': 16,
  'average_dump_ins_made': 2.2857142857142856,
  'average_dump_ins_percentage': 0.21428571428571425,
  'overall_dump_ins_percentage': 0.3333333333333333,
  'to